**Title:** DistilBERT for Text Classification

**Introduction:**
This notebook demonstrates the implementation of a DistilBERT model for text classification using PyTorch and the Hugging Face Transformers library. DistilBERT is a smaller, faster, and lighter version of the BERT model while retaining most of its performance. The dataset used in this notebook consists of labeled tweets, and the objective is to classify the tweets into three categories: Hate Speech, Offensive Language, and Neither.

**Content:**

1. **Environment Setup:** The notebook begins by setting up the Python environment and importing necessary libraries, including PyTorch, Transformers, and scikit-learn.

2. **Text Cleaning:** A function for cleaning text data is defined to preprocess the tweets, removing URLs, mentions, special characters, and emoticons.

3. **Dataset Loading and Splitting:** The labeled tweet dataset is loaded and split into training, validation, and test sets.

4. **Tokenization with DistilBERT Tokenizer:** The tweets are tokenized using the DistilBERT tokenizer, which converts text inputs into token IDs and attention masks.

5. **Dataset Class:** A custom dataset class is defined to process the tokenized inputs and corresponding labels.

6. **DataLoader Initialization:** DataLoaders are initialized for the training, validation, and test datasets to efficiently load data in batches during model training and evaluation.

7. **Model Initialization:** The DistilBERT model for sequence classification (`DistilBertForSequenceClassification`) is initialized with the pre-trained weights from the `distilbert-base-uncased` model. The model is then moved to the appropriate device (GPU or CPU).

8. **Training Function:** A training function is defined to train the DistilBERT model on the training dataset using backpropagation.

9. **Evaluation Function:** An evaluation function is defined to assess the model's performance on the validation and test datasets.

10. **Main Training Loop:** The main training loop runs for a specified number of epochs, during which the model is trained on the training dataset and evaluated on the validation dataset.

11. **Final Evaluation on Test Set:** The trained model's performance is evaluated on the test dataset, and classification metrics such as precision, recall, and F1-score are computed.

12. **Conclusion:** The notebook concludes by printing the test accuracy of the DistilBERT model for text classification.

**Conclusion:**
This notebook provides a complete implementation of a DistilBERT model for text classification using PyTorch and the Hugging Face Transformers library. DistilBERT offers a more computationally efficient alternative to BERT while maintaining competitive performance. The notebook demonstrates the effectiveness of leveraging pre-trained models like DistilBERT for NLP tasks, making it a valuable tool in natural language processing applications.

In [12]:
import re
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


In [13]:
emoticons = [':-)', ':)', '(:', '(-:', ':))', '((:', ':-D', ':D', 'X-D', 'XD', 'xD', 'xD', '<3', '3', ':*', ':-*', 'xP', 'XP', 'XP', 'Xp', ':-|', ':->', ':-<', '8-)', ':-P', ':-p', '=P', '=p', ':*)', '*-*', 'B-)', 'O.o', 'X-(', ')-X']

def clean_text(text):
    text = text.lower()
    text = re.sub(r'https?://[^\s]+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'\d+', '', text)
    for emoticon in emoticons:
        text = text.replace(emoticon, '')
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)
    text = re.sub(r"([?.!,¿])", r" ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text.strip()


In [14]:
# Load dataset
df = pd.read_csv('/kaggle/input/dataset/labeled_data.csv')
df['tweet'] = df['tweet'].apply(clean_text)

# Split dataset
train_texts, temp_texts, train_labels, temp_labels = train_test_split(df['tweet'], df['class'], test_size=0.3, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, random_state=42)


In [15]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=128)


In [16]:
# Dataset class
class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels.tolist())
test_dataset = TweetDataset(test_encodings, test_labels.tolist())
val_dataset = TweetDataset(val_encodings, val_labels.tolist())

In [17]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [18]:
# Model initialization
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)  # Adjust num_labels according to your classification problem
optimizer = AdamW(model.parameters(), lr=5e-6)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training function
def train(epoch):
    model.train()
    total_loss, total_accuracy = 0, 0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch}"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        logits = outputs.logits.detach().cpu().numpy()
        predictions = np.argmax(logits, axis=-1)
        total_accuracy += accuracy_score(labels.cpu().numpy(), predictions)
    
    avg_loss = total_loss / len(train_loader)
    avg_accuracy = total_accuracy / len(train_loader)
    print(f"Training Loss: {avg_loss:.3f}")
    print(f"Training Accuracy: {avg_accuracy:.3f}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
# Evaluation function
def evaluate(loader, desc="Evaluating"):
    model.eval()
    total_loss, total_accuracy = 0, 0
    all_predictions, all_labels = [], []
    
    for batch in tqdm(loader, desc=desc):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        loss = outputs.loss.item()
        total_loss += loss
        logits = outputs.logits.detach().cpu().numpy()
        predictions = np.argmax(logits, axis=-1)
        total_accuracy += accuracy_score(labels.cpu().numpy(), predictions)
        
        all_predictions.extend(predictions)
        all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(loader)
    avg_accuracy = total_accuracy / len(loader)
    print(f"Validation Loss: {avg_loss:.3f}")
    print(f"Validation Accuracy: {avg_accuracy:.3f}")
    
    return all_labels, all_predictions

In [21]:
# Main training loop
for epoch in range(1, 4):
    train(epoch)
    evaluate(val_loader)

# Final evaluation on test set
labels, predictions = evaluate(test_loader, "Final Test Evaluation")
print(classification_report(labels, predictions, target_names=['Hate Speech', 'Offensive Language', 'Neither']))

# Accuracy
accuracy = accuracy_score(labels, predictions)
print(f"Test Accuracy: {accuracy:.3f}")

Training Epoch 1: 100%|██████████| 543/543 [00:47<00:00, 11.49it/s]


Training Loss: 0.408
Training Accuracy: 0.860


Evaluating: 100%|██████████| 117/117 [00:03<00:00, 35.95it/s]


Validation Loss: 0.291
Validation Accuracy: 0.899


Training Epoch 2: 100%|██████████| 543/543 [00:47<00:00, 11.50it/s]


...

Training Loss: 0.221
Training Accuracy: 0.923


Evaluating: 100%|██████████| 117/117 [00:03<00:00, 36.08it/s]


Validation Loss: 0.245
Validation Accuracy: 0.915


Final Test Evaluation: 100%|██████████| 117/117 [00:03<00:00, 36.93it/s]

Validation Loss: 0.233
Validation Accuracy: 0.914
                    precision    recall  f1-score   support

       Hate Speech       0.51      0.23      0.32       207
Offensive Language       0.93      0.96      0.95      2880
           Neither       0.88      0.91      0.89       631

          accuracy                           0.91      3718
         macro avg       0.77      0.70      0.72      3718
      weighted avg       0.90      0.91      0.90      3718

Test Accuracy: 0.913
